# Extracting wages from unstructured job postings using transformer
Author: Kexin Feng

This project aims to extract wage information from unstructured job postings. The job posting data is from [LinkUp](https://www.linkup.com/), a platform that aggregates job listings directly from company websites.

I use a three-step approach for wage extraction: 

1. Isolating Text Segments Containing Wage Data
2. **Fine-Tuning a Question-Answering Transformer Model**
3. Extracting and Categorizing Wage Figures Using Regular Expressions

This Jupyter notebook shows the second step process - finetuning a transformer using labeled data. The findings and analyses using the extracted wage data are available in the working paper ["Pay Transparency and Gender Differences in Job Search"](https://dx.doi.org/10.2139/ssrn.4984519).

## Step 1: Prepare the training set
Job postings from LinkUp are typically lengthy, and over 90\% of them do not contain any wage information. Processing these extensive texts without any filtration would be highly time- and resource-consuming. To streamline the finetuning process, I extracted text segments from job descriptions that contain a dollar sign followed by digits (e.g., $\$$12, $\$$9, $\$$52,000). This targeted extraction focuses on sentences likely to include wage information.

This method may miss instances where wage information is presented without a dollar sign, such as "salary range: 120000 - 160000." However, such cases are relatively rare and represent only a small fraction of the dataset.


In [1]:
import pandas as pd
from transformers import pipeline
import re
import os
import numpy as np
import glob
import torch

pd.options.display.float_format = '{:.3f}'.format # adjust the display format
path = "/mnt/home/fengkexi/wage_compression/cleaned_data"
os.chdir(path)

/mnt/home/fengkexi/anaconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


First we load the training set and keep only the relevant columns.

In [20]:
# load training set
df = pd.read_csv('linkup_10000sample.csv').reset_index(drop=True)
df = df[['sent', 'answer']]
# df.rename(columns={"hash": "id"}, inplace=True)

In [21]:
print(df)

                                                   sent             answer
0     Headquartered in Reston, Virginia, Leidos repo...                NaN
1     Headquartered in Reston, Virginia, Leidos repo...                NaN
2     Headquartered in Reston, Virginia, Leidos repo...                NaN
3     Headquartered in Reston, Virginia, Leidos repo...                NaN
4     Headquartered in Reston, Virginia, Leidos repo...                NaN
...                                                 ...                ...
9995  Position Description Title/PD#: Financial Mana...  $79,488  $122,875
9996  Work Schedule: Monday through Friday, 7:00am t...   $48,987  $99,104
9997  Work Schedule: Monday - Friday, 8:00 AM - 4:30...   $61,218  $79,586
9998  With more than 2,500 employees, Charleston VA ...   $41,365  $53,773
9999  Work Schedule: Monday - Friday, 8:00 a.m. - 4:...  $94,046  $125,077

[10000 rows x 2 columns]


Next we rename the column 'sent' to 'context' to clarify that this column contains the text context needed for the transformer model.

In [22]:
df.rename(columns={"sent": "context"}, inplace=True)

Preprocess the text data: Convert all text data to lowercase to standardize the dataset and ensure they are of string type.

In [23]:
df['answer'] = df['answer'].str.lower()
df['context'] = df['context'].str.lower()

df['context'] = df['context'].astype(str)
df['answer'] = df['answer'].astype(str)

We now compute the starting position of each answer within its respective context, which is crucial for question answering models that need the exact location of answers in the text.


In [24]:
for index, row in df.iterrows():
    Answer = row['answer']
    Context = row['context']
    Start_pos = Context.find(Answer)
    df.at[index, 'start_pos'] = Start_pos
    
df['start_pos'] = df['start_pos'].astype(int)

In [25]:
print(df)

                                                context             answer  \
0     headquartered in reston, virginia, leidos repo...                nan   
1     headquartered in reston, virginia, leidos repo...                nan   
2     headquartered in reston, virginia, leidos repo...                nan   
3     headquartered in reston, virginia, leidos repo...                nan   
4     headquartered in reston, virginia, leidos repo...                nan   
...                                                 ...                ...   
9995  position description title/pd#: financial mana...  $79,488  $122,875   
9996  work schedule: monday through friday, 7:00am t...   $48,987  $99,104   
9997  work schedule: monday - friday, 8:00 am - 4:30...   $61,218  $79,586   
9998  with more than 2,500 employees, charleston va ...   $41,365  $53,773   
9999  work schedule: monday - friday, 8:00 a.m. - 4:...  $94,046  $125,077   

      start_pos  
0            -1  
1            -1  
2        

remove any rows where the answer cannot be found in the context unless the answer is 'nan', which indicates a missing or irrelevant answer-these texts do not contain any wage information.


In [28]:
condition = (df['start_pos'] == -1) & (df['answer'] != "nan")
df = df[~(condition)] 

In [29]:
print(df)

                                                context             answer  \
0     headquartered in reston, virginia, leidos repo...                nan   
1     headquartered in reston, virginia, leidos repo...                nan   
2     headquartered in reston, virginia, leidos repo...                nan   
3     headquartered in reston, virginia, leidos repo...                nan   
4     headquartered in reston, virginia, leidos repo...                nan   
...                                                 ...                ...   
9995  position description title/pd#: financial mana...  $79,488  $122,875   
9996  work schedule: monday through friday, 7:00am t...   $48,987  $99,104   
9997  work schedule: monday - friday, 8:00 am - 4:30...   $61,218  $79,586   
9998  with more than 2,500 employees, charleston va ...   $41,365  $53,773   
9999  work schedule: monday - friday, 8:00 a.m. - 4:...  $94,046  $125,077   

      start_pos  
0            -1  
1            -1  
2        

For any 'nan' answers, which were from I make the answer column in the string type, we set the 'start_pos' to -1 to indicate no valid starting position.


In [30]:
df.loc[df['answer'] == "nan", 'start_pos'] = -1

Here, we create a bnew 'answers' column to contain a dictionary with the text of the answer and its starting position, tailored to the input needs of transformer-based models that require structured input for training.

In [31]:
# Creating a new column 'answers' with structured data for model training
df['answers'] = df.apply(
    lambda row: {
        'text': [row['answer']] if row['answer'] != 'nan' else [],
        'answer_start': [row['start_pos']] if row['start_pos'] != -1 else []
    },
    axis=1
)

In [ ]:
print(df)

A question-answering transformer requires two inputs to extract an answer: a question and a context. I already have the context, and I need the question now. After some playing with transformer models, I choose the question to be "What is the salary range" and add a question column to the dataset, which seems to work fine across contexts.

In [ ]:
df['question'] = 'What is the salary range' # Add a Question Column

Now we have successfully loaded and processed the training dataset, making it ready for fine-tuning a transformer model. 

## Step 2: Finetune a question answering tranformer using HuggingFace

The code I use for this step is based on the code from this link: https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/question_answering.ipynb.

Now we will fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a question answering task, which is the task of extracting the answer to a question from a given context (job posting).

![Widget inference representing the QA task](https://github.com/huggingface/notebooks/blob/main/examples/images/question_answering.png?raw=1)

**Note:** This notebook finetunes models that answer question by taking a substring of a context, not by generating new text.

There are many pretrained question answering transformer models to choose on Hugging Face. I tested several pretrained models from Huggingface (in early 2023) and deberta-v3-large-squad2 has the best baseline performance, which naturally becomes the model to finetune.


This notebook is built to run on any question answering task with the same format as SQUAD (version 1 or 2), with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [6]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not). which allows for possible questions with no answer.
squad_v2 = True


model_checkpoint = "deepset/deberta-v3-large-squad2"

batch_size = 8 # select batch size based on the model and GPU

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the function `load_metric`. 

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from datasets import load_dataset, load_metric

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Now one specific thing for the preprocessing in question answering is how to deal with very long documents. We usually truncate them in other tasks, when they are longer than the model maximum sentence length, but here, removing part of the the context might result in losing the answer we are looking for. To deal with this, we will allow one (long) example in our dataset to give several input features, each of length shorter than the maximum length of the model (or the one we set as a hyper-parameter). Also, just in case the answer lies at the point we split a long context, we allow some overlap between the features we generate controlled by the hyper-parameter `doc_stride`:

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

Split the labeled dataset into training set and test set with a 85/15 split.

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# set the fraction of data to be used for validation
validation_size = 0.15

# split the dataset into training and validation sets
train_dataset, validation_dataset = train_test_split(df, test_size=validation_size)

# check the number of examples in each split
print(f"Number of training examples: {len(train_dataset)}")
print(f"Number of validation examples: {len(validation_dataset)}")

The following transformation converts pandas DataFrames into Hugging Face Dataset objects, organizing them into a DatasetDict for easier handling during training.

In [ ]:
# Convert DataFrame to Dataset

train_dataset = Dataset.from_pandas(train_dataset)
valid_dataset = Dataset.from_pandas(validation_dataset)

# create the DatasetDict
datasets = DatasetDict({'train': train_dataset, 'validation': valid_dataset})
print(datasets)

For this notebook to work with any kind of models, we need to account for the special case where the model expects padding on the left (in which case we switch the order of the question and the context):

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

Now let's put everything together in one function we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag `allow_impossible_answers` is `False`. Since the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    print("sample_mapping_is" , sample_mapping)
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")
    print("offset_mapping_is" , offset_mapping)


    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            try:
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])
            except:
                print("hahahah")
                print(answers)
                

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `AutoModelForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-jobposting",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    #learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_steps = 50,
    gradient_accumulation_steps = 16
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-squad"` or `"huggingface/bert-finetuned-squad"`).

Then we will need a data collator that will batch our processed examples together, here the default one will work:

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

We will evaluate our model and compute metrics in the next section (this is a very long operation, so we will only compute the evaluation loss during training).

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"], # validation
    data_collator=data_collator,
    tokenizer=tokenizer,
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train(resume_from_checkpoint=False)

Since this training is particularly long, let's save the model just in case we need to restart.

In [ ]:
trainer.save_model("jobposting-trained")

## Evaluation
Evaluating the finetuned model will require a bit more work, as we will need to map the predictions of our model back to parts of the context. The model itself predicts logits for the start and en position of our answers: if we take a batch from our validation datalaoder, here is the output our model gives us:

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

And then we can sort the `valid_answers` according to their `score` and only keep the best one. The only point left is how to check a given span is inside the context (and not the question) and how to get back the text inside. To do this, we need to add two things to our validation features:
- the ID of the example that generated the feature (since each example can generate several features, as seen before);
- the offset mapping that will give us a map from token indices to character positions in the context.

That's why we will re-process the validation set with the following function, slightly different from `prepare_train_features`:

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

And like before, we can apply that function to our validation set easily:

In [ ]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Now we can grab the predictions for all features by using the `Trainer.predict` method:

In [189]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set don't have a corresponding argument in `DebertaV2ForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `DebertaV2ForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3302
  Batch size = 8


The `Trainer` *hides* the columns that are not used by the model (here `example_id` and `offset_mapping` which we will need for our post-processing), so we set them back:

In [190]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

We can now refine the test we had before: since we set `None` in the offset mappings when it corresponds to a part of the question, it's easy to check if an answer is fully inside the context. We also eliminate very long answers from our considerations (with an hyper-parameter we can tune)

In [191]:
max_answer_length = 30

In [192]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 18.02921, 'text': ' salary range: $70,171 to $124,020 / per year'},
 {'score': 16.397919, 'text': ' $70,171 to $124,020 / per year'},
 {'score': 11.685001, 'text': ' salary range: $70,171 to $124,020'},
 {'score': 11.337915, 'text': ' range: $70,171 to $124,020 / per year'},
 {'score': 11.114033,
  'text': ' salary range: $70,171 to $124,020 / per year open'},
 {'score': 10.053709, 'text': ' $70,171 to $124,020'},
 {'score': 9.58847, 'text': ' salary range: $70,171 to $124,020 / per'},
 {'score': 9.565637, 'text': ' salary range: $70,171 to $124,020 /'},
 {'score': 9.494775, 'text': ' salary'},
 {'score': 9.48274, 'text': ' $70,171 to $124,020 / per year open'},
 {'score': 8.478324, 'text': ' salary range:'},
 {'score': 8.258965, 'text': '70,171 to $124,020 / per year'},
 {'score': 8.207971, 'text': ' $124,020 / per year'},
 {'score': 8.137117, 'text': ' salary range'},
 {'score': 8.133571, 'text': ' year'},
 {'score': 7.9571786, 'text': ' $70,171 to $124,020 / per'},
 {'sco

We can compare to the actual ground-truth answer:

In [193]:
datasets["validation"][0]["answers"]

{'answer_start': [595], 'text': ['$70,171 to $124,020 / per year']}

Our model picked the right answer!

As we mentioned in the code above, this was easy on the first feature because we knew it comes from the first example. For the other features, we will need a map between examples and their corresponding features. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices:

In [194]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

We're almost ready for our post-processing function. The last bit to deal with is the impossible answer (when `squad_v2 = True`). The code above only keeps answers that are inside the context, we need to also grab the score for the impossible answer (which has start and end indices corresponding to the index of the CLS token). When one example gives several features, we have to predict the impossible answer when all the features give a high score to the impossible answer (since one feature could predict the impossible answer just because the answer isn't in the part of the context it has access too), which is why the score of the impossible answer for one example is the *minimum* of the scores for the impossible answer in each feature generated by the example.

We then predict the impossible answer when that score is greater than the score of the best non-impossible answer. All combined together, this gives us this post-processing function:

In [195]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

And we can apply our post-processing function to our raw predictions:

In [196]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 2694 example predictions split into 3302 features.


  0%|          | 0/2694 [00:00<?, ?it/s]

Then we can load the metric from the datasets library.

In [197]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

/tmp/local/7580721/ipykernel_139211/2905994612.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")


Then we can call compute on it. We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary. In the case of squad_v2, we also have to set a `no_answer_probability` argument (which we set to 0.0 here as we have already set the answer to empty if we picked it).

In [198]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]#validation
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 88.0475129918337,
 'f1': 93.76478549084337,
 'total': 2694,
 'HasAns_exact': 82.62857142857143,
 'HasAns_f1': 91.42990406418981,
 'HasAns_total': 1750,
 'NoAns_exact': 98.09322033898304,
 'NoAns_f1': 98.09322033898304,
 'NoAns_total': 944,
 'best_exact': 88.0475129918337,
 'best_exact_thresh': 0.0,
 'best_f1': 93.76478549084341,
 'best_f1_thresh': 0.0}

Hooray! The finetuned model achieves an exact match rate of 88.05% and an F1 score of 93.76%, which is fantastic and good enough for the subsequent research.

In [207]:
# Evaluate the original model before finetuning 
from evaluate import evaluator
from datasets import load_dataset
task_evaluator = evaluator("question-answering")
results = task_evaluator.compute(
    model_or_pipeline="deepset/deberta-v3-base-squad2",
    data=datasets['validation'],
    metric="squad_v2",
    squad_v2_format=True,
)

loading configuration file config.json from cache at /mnt/home/fengkexi/.cache/huggingface/hub/models--deepset--deberta-v3-base-squad2/snapshots/9258f36d79eb17f9ccf2b3bcae1ce8952e8da76d/config.json
Model config DebertaV2Config {
  "_name_or_path": "deepset/deberta-v3-base-squad2",
  "architectures": [
    "DebertaV2ForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "language": "english",
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "name": "DebertaV2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_a

In [208]:
results # metrics 

{'exact': 54.083147735708984,
 'f1': 66.39089117718106,
 'total': 2694,
 'HasAns_exact': 49.48571428571429,
 'HasAns_f1': 68.43260618932885,
 'HasAns_total': 1750,
 'NoAns_exact': 62.605932203389834,
 'NoAns_f1': 62.605932203389834,
 'NoAns_total': 944,
 'best_exact': 54.083147735708984,
 'best_exact_thresh': 0.9998341798782349,
 'best_f1': 66.39089117718108,
 'best_f1_thresh': 0.9998341798782349,
 'total_time_in_seconds': 47.13784034643322,
 'samples_per_second': 57.151536434440125,
 'latency_in_seconds': 0.017497342370613667}

The original model, before finetuning, has an exact match rate of 54.08% and F1 score of 66.39%, respectively. The finetuned model significantly outperforms the base model.